### Notebook for constructiong POD-NODE NIROM approximation for a flow around a cylinder example

A collection of high-fidelity snapshots are generated that sufficiently capture the time-dynamics of the simulation. POD is adopted to define a reduced basis space for the high-fidelity snaphosts. The evolution of the time dynamics in the POD-latent space is modeled using Neural ODEs (NODE).  

Adaptive Hydraulics (AdH) is used as the high-fidelity model for simulating two real world flow examples governed by the 2d depth-averaged shallow water equations.

#### Note
This notebook serves as an example of how to set up and evaluate a PODNODE model for the given dataset. However, in order to attain a desirable level of prediction accuracy, the training time is high. Please refer to 
```
S. Dutta, P. Rivera-casillas, and M. W. Farthing, “Neural Ordinary Differential Equations for Data-Driven Reduced Order Modeling of Environmental Hydrodynamics,” in Proceedings of the AAAI 2021 Spring Symposium on Combining Artificial Intelligence and Machine Learning with Physical Sciences, 2021. 
arXiv:2104.13962 [cs.LG]
```
for model configuration details.

In [ ]:
### Loading modules
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import scipy
import os
import gc
import argparse
import platform
print("Python "+str(platform.python_version()))
import importlib
from importlib import reload as reload

import tensorflow as tf
print("Tensorflow "+ str(tf.__version__))
if tf.__version__ == '1.15.0':
    tf.compat.v1.enable_eager_execution()
elif tf.__version__.split('.')[0] == 2: 
    print("Setting Keras backend datatype")
    tf.keras.backend.set_floatx('float64')

from tfdiffeq import odeint,odeint_adjoint
from tfdiffeq.adjoint import odeint as adjoint_odeint
tf.keras.backend.set_floatx('float64')
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
np.random.seed(0)

basedir   = os.getcwd()
srcdir = os.path.join(basedir,'../pynirom/')
workdir   = os.path.join(basedir,'../examples/')
datadir   = os.path.join(basedir,'../data/')
figdir    = os.path.join(basedir,'../figures/podnode')
nodedir   = os.path.join(basedir,'../data/')

import pynirom
from pynirom.pod import pod_utils as pod
from pynirom.utils import data_utils as du
from pynirom.node import main as nd
from pynirom.node import plotting as pu
from pynirom.node import node as node



In [ ]:
device = 'cpu:0'           # select gpu:# or cpu:#
purpose= 'retrain'           #'train' to train a new model, 
                           # 'retrain' to start training from an existing model, and 
                           # 'eval' to load a pre-trained model for evaluation 
stacking = True            #If True, Specify new stacking order of latent space vector
stack_order = 'S_dep,S_vx,S_vy'  #If stacking = True, specify the stacking order of the latent space vector
scale_time = True          #Scale time or not (Normalize)
scale_states = False       #Scale states 
scaling_method = 'maxabs'  #Scaling method: 'centered', 'minmax' or 'maxabs'
augmented,aug_dims = (False,5) #Augmented or not and size of augmentation
N_layers = int(1)          #Only four layers supported as of now.
N_neurons = int(256)       #Number of neurons per layer
act_f = 'linear'             #Activation Function ('linear', 'tanh', 'sigmoid',...), default='tanh'
learning_rate_decay = True #Use decaying learning rate or not
initial_learning_rate = float(0.001) #If 'learning_rate_decay = False' then this is the fixed learning rate
decay_steps = int(5001)    #Number of steps for learning rate decay
decay_rate = float(0.5)    #Rate of learning rate decay
staircase_opt = True       #True for staircase decay and False for exponential
optimizer = 'RMSprop'      #See pynirom.node.node.set_optimizer() for options
use_adjoint = False        #Use adjoint method or not
solver = 'rk4'             #Specify ODE solver. See tfdiffeq README for available options 
use_minibatch, batch_size = (False,64) #Use minibatch or not and batch size
epochs = int(100)           #Number of epochs of training


print("\n***** Runtime parameters: ******\n")
print(f'Mode = {purpose}, Scaling = {scale_states}, Augmenting = {augmented}, Adjoint = {use_adjoint}')
print(f'Solver = {solver}, Optimizer = {optimizer}, Stacking order = {stack_order}, Epochs = {epochs}')
print(f'# Layers = {N_layers}, # Neurons per layer = {N_neurons}, Activation fn = {act_f}')
if use_minibatch:
    print(f'Use minibatch = {use_minibatch}, Batch size = {batch_size}')
if learning_rate_decay:
    print(f'Init LR = {initial_learning_rate}, # LR decay steps = {decay_steps}, LR decay rate = {decay_rate}')
else:
    print(f'Fixed LR = {initial_learning_rate}')
print('**********************************\n')



In [ ]:
### ------ Import Snapshot data -------------------
model_sw = 'RED'

if model_sw =='SD':
    data = np.load(datadir + 'san_diego_tide_snapshots_T4.32e5_nn6311_dt25.npz')
    mesh = np.load(datadir + 'san_diego_mesh.npz')
elif model_sw == 'RED':
    data = np.load(datadir + 'red_river_inset_snapshots_T7.0e4_nn12291_dt10.npz')
    mesh = np.load(datadir + 'red_river_mesh.npz')

savedir = nodedir+model_sw+'/current'
pre_trained_dir = savedir+'/model_weights/' #If 'eval' specify path for pretrained model

print('HFM data has {0} snapshots of dimension {1} for h,u and v, spanning times [{2}, {3}]'.format(
                    data['T'].shape[0],data['S_dep'].shape[0],
                    data['T'][0], data['T'][-1]))


## ------- Prepare training snapshots ----------------
print('\n-------Prepare training and testing data---------')
soln_names = ['S_dep', 'S_vx', 'S_vy']
nodes = mesh['nodes'];  
triangles = mesh['triangles']; 

snap_start = 100
if model_sw == 'SD':
    T_end = 50*3600   ### 50 hours in seconds
    snap_incr = 4
elif model_sw == 'RED':
    T_end = 3.24e4    ### 9 hours in seconds
    snap_incr = 3

snap_train, times_train = du.prepare_data(data, soln_names, start_skip=snap_start, T_end=T_end, incr=snap_incr)
print('Using {0} training snapshots for time interval [{1:.3f},{2:.3f}] hours'.format(times_train.shape[0],
                                        times_train[0]/3600, times_train[-1]/3600))

## ------- Prepare testing snapshots ----------------
pred_incr = snap_incr - 2
snap_pred_true, times_predict = du.prepare_data(data, soln_names, start_skip=snap_start, T_end=T_end, 
                                                    incr=pred_incr)
print('Using {0} testing snapshots for time interval [{1:.3f},{2:.3f}] hours'.format(times_predict.shape[0],
                                        times_predict[0]/3600, times_predict[-1]/3600))



del data
del mesh
gc.collect()

In [ ]:
### ------ Compute the POD basis using the training snapshots------------------
if model_sw == 'SD':
    trunc_lvl = 0.9999995  
elif model_sw == 'RED':
    trunc_lvl = 0.99
    
snap_norm, snap_mean, U, D, W = pod.compute_pod_multicomponent(snap_train)
nw, U_r = pod.compute_trunc_basis(D, U, eng_cap = trunc_lvl)

### ------ Compute the POD coefficients for training snapshots------------------
Z_train = pod.project_onto_basis(snap_train, U_r, snap_mean)


### ------ Compute the POD coefficients for the truth snapshots on the prediction interval------------------
Z_pred_true = pod.project_onto_basis(snap_pred_true, U_r, snap_mean)

npod_total = 0
for key in soln_names:
    npod_total+=nw[key]

In [ ]:
### ---- Setup NODE input data

NODE = nd.NODEBase(device=device)
true_state_array, true_pred_state_array, init_state, state_len, dt_train, dt_predict = \
        NODE.prepare_input_data(Z_train, nw, times_train, stack_order, times_predict, Z_pred_true)

print("Training NODE using %d modes for %d time steps with %.3f <= t <= %.3f and dt = %.4f"%(state_len,
                             true_state_array.shape[0], times_train[0], 
                             times_train[-1], dt_train))
print("Predicting NODE solutions using %d modes for %d time steps with %.3f <= t <= %.3f and dt = %.4f"%(
                            state_len, true_pred_state_array.shape[0], times_predict[0], 
                            times_predict[-1], dt_predict))

In [ ]:
### Preprocess training data (scale time and/or states, augment states if using ANODE)
### Set up learning rate scheduler and optimizer for training of the NODE model

true_state_tensor, times_tensor, init_tensor, learn_rate, optim = \
                NODE.preprocess_data(scale_states=scale_states, augmented=augmented, 
                        lr_decay=learning_rate_decay, init_lr=initial_learning_rate, opt=optimizer, 
                        scaling_method=scaling_method, aug_dim=aug_dims, 
                        decay_steps=decay_steps, decay_rate=decay_rate, staircase=staircase_opt, )

In [ ]:
### ---- Model Training ------

train_loss_results, train_lr, saved_ep = \
         NODE.train_model(true_state_tensor, times_tensor, init_tensor, epochs, savedir,
                    solver=solver, purpose=purpose, adjoint=use_adjoint, minibatch=use_minibatch,
                         pre_trained_dir = pre_trained_dir)


In [ ]:
## --- Generate NODE predictions ---

predicted_states, times_predict = NODE.predict_time(times_predict, init_tensor, pre_trained_dir,)

## ---- Compute Mean Square Error of predictions
Z_pred = {}
ctr= 0
for key in stack_order.split(','):
    Z_pred[key] = np.array(predicted_states)[:,ctr:ctr+nw[key]].T
    ctr += nw[key]
snap_pred = pod.reconstruct_from_rom(Z_pred, U_r, snap_mean, nw)

error_h = np.mean(np.square(snap_pred['S_dep']-snap_pred_true['S_dep']))
error_vx = np.mean(np.square(snap_pred['S_vx']-snap_pred_true['S_vx']))
error_vy = np.mean(np.square(snap_pred['S_vy']-snap_pred_true['S_vy']))

print("\n---- Mean Square Error of NODE predictions ----\n")
print('H MSE: ' + str(error_h))
print('Vx MSE: ' + str(error_vx))
print('Vy MSE: ' + str(error_vy))


In [ ]:
def set_label(key):
    if key == 'S_vx':
        return 'u'
    elif key == 'S_vy':
        return 'v'
    elif key == 'S_dep':
        return 'h'

In [ ]:
### ----- Visualize true and predicted POD coefficients -------

comp = 0

# Visualization fluff here
fig, ax = plt.subplots(nrows=3,ncols=1,figsize=(8,15))
mnum = comp
for i, key in enumerate(soln_names):
    tt = ax[i].plot(times_predict[:],true_pred_state_array[:,mnum],label='True',marker='o',markevery=20)
    # Visualization of modal evolution using NODE
    ln, = ax[i].plot(times_predict[:],predicted_states[:,mnum],label='NODE',color='orange',marker='D',
                     markevery=25)
    mnum = mnum + nw[key]

    ax[i].set_xlabel('Time', fontsize=18)
    sv = set_label(key)+', mode '+str(comp)
    ax[i].set_ylabel(sv,fontsize=18)
    ax[i].legend(fontsize=14)
fig.suptitle("POD coefficients of the HFM and NODE solutions", fontsize=20)
fig.tight_layout(rect=[0, 0.03, 1, 0.98])

In [ ]:
## ---- Compute spatial RMS/Relative error

metric = 'rms'
err = NODE.compute_error(snap_pred_true, snap_pred, soln_names, metric=metric)

vstring = {}
for key in soln_names:
    vstring[key] = set_label(key)

## ---- Visualize computed error metric
pu.plot_NODE_err(err, times_predict/3600, soln_names, vstring, metric=metric, unit='hours')

In [ ]:
#### ----- Save predicted solutions -------
save_nirom_solutions = False

if save_nirom_solutions:
    os.chdir(nodedir)
    print("Saving results in %s"%(os.getcwd()))
    
#     if model_sw == 'RED':
#         model = 'Red'
#     elif model_sw == 'SD':
#         model = 'SD'
    
    np.savez_compressed('%s_online_node'%(model_sw), S_dep=snap_pred['S_dep'],S_vx = snap_pred['S_vx'], 
                        S_vy = snap_pred['S_vy'], time=times_predict, loss=train_loss_results,)

